<a href="https://colab.research.google.com/github/roquearmas/IABD_DATA/blob/main/reinforcement_learning_deep_q_learning_cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalación de librerías necesarias

In [ ]:
!pip install gym
!pip install pygame
!pip install tensorflow

Importo las librerías que necesita el código

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gym # for environment
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam # adaptive momentum
import random
import pygame
import pickle
from tensorflow.keras.models import load_model


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

La ecuación de Bellman, utilizada en el metodo ***Replay*** es importante en el campo del aprendizaje por refuerzo. Describe cómo se relaciona la función de valor de un estado y acción con las recompensas inmediatas y futuras. En el caso de la función de valor  Q(s, a) para un estado $(s)$ y una acción $(a)$, la ecuación de Bellman se expresa de la siguiente manera:

\begin{equation}
Q(s, a) = R(s, a) + \gamma \cdot \max_{a'} Q(s', a')
\end{equation}

Donde:

- $Q(s, a)$  es la función de valor para el estado $(s)$ y la acción $(a)$.
- $R(s, a)$  representa la recompensa inmediata al tomar la acción $(a)$ en el estado $(s)$.

- $\gamma$ es el factor de descuento que pondera las recompensas futuras.


- $( max_{a'} Q(s', a') )$ denota el valor máximo de la función de valor para el próximo estado $(s')$ considerando todas las posibles acciones $(a')$.

Esta ecuación es fundamental en el cálculo de la función de valor óptima en el aprendizaje por refuerzo, ya que establece la relación entre las recompensas presentes y futuras en la toma de decisiones óptimas. \

En el siguiente bloque se genera el agente y se entrena un modelo de Tensorflow de Keras, con una hiperparametrización dada, específica para resolver el problema de [CartPole de Pygame](https://pylessons.com/CartPole-reinforcement-learning)

In [8]:

class DQLAgent():

    def __init__(self, env):
        # parameters and hyperparameters

        # this part is for neural network or build_model()
        self.state_size = env.observation_space.shape[0] # this is for input of neural network node size
        self.action_size = env.action_space.n # this is for out of neural network node size

        # this part is for replay()
        self.gamma = 0.95
        self.learning_rate = 0.001

        # this part is for adaptiveEGreedy()
        self.epsilon = 1 # initial exploration rate
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01

        self.memory = deque(maxlen = 1000) # a list with 1000 memory, if it becomes full first inputs will be deleted

        #self.model = self.build_model()
        self.model =self.carga_modelo()

    def build_model(self):
        # neural network for deep Q learning
        model = Sequential()
        model.add(Dense(24, input_dim = self.state_size, activation = 'relu'))
        model.add(Dense(24, activation ='relu'))
        #model.add(Dense(48, input_dim = self.state_size, activation = 'tanh')) # first hidden layer
        model.add(Dense(self.action_size, activation = 'linear')) # output layer
        model.compile(loss = 'mse', optimizer = Adam(learning_rate=self.learning_rate))

        return model

    def carga_modelo (self):

        model = load_model('modelo_cartpole_entrenado_relu.h5')
        return model

    def remember(self, state, action, reward, next_state, done):
        # storage
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        # acting, exploit or explore
        if random.uniform(0,1) <= self.epsilon:
            return env.action_space.sample()
        else:
            act_values = self.model.predict(state)
            return np.argmax(act_values[0])


    def replay(self, batch_size):
        # training

        if len(self.memory) < batch_size:
            return # memory is still not full

        minibatch = random.sample(self.memory, batch_size) # take 16 (batch_size) random samples from memory
        for state, action, reward, next_state, done in minibatch:
            if done: # if the game is over, I dont have next state, I just have reward
                target = reward
            else:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
                # target = R(s,a) + gamma * max Q`(s`,a`)
                # target (max Q` value) is output of Neural Network which takes s` as an input
                # amax(): flatten the lists (make them 1 list) and take max value
            train_target = self.model.predict(state) # s --> NN --> Q(s,a)=train_target
            train_target[0][action] = target
            self.model.fit(state, train_target, verbose = 0) # verbose: dont show loss and epoch

    def adaptiveEGreedy(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


if __name__ == "__main__":

    # initialize gym environment and agent
    env = gym.make('CartPole-v0')
    agent = DQLAgent(env)

    batch_size = 16
    episodes = 10
    for e in range(episodes):

        # initialize environment
        state = env.reset()
        state = np.reshape(state, [1,4])

        time = 0 # each second I will get reward, because I want to sustain a balance forever
        while True:

            # act
            action = agent.act(state)

            # step
            next_state, reward, done, _ = env.step(action)
            next_state = np.reshape(next_state, [1,4])

            # remember / storage
            agent.remember(state, action, reward, next_state, done)

            # update state
            state = next_state

            # replay
            agent.replay(batch_size)

            # adjust epsilon
            agent.adaptiveEGreedy()

            time += 1

            if done:
                print('episode: {}, time: {}'.format(e, time))
                break


/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated,

Se han truncado las últimas 5000 líneas del flujo de salida.
episode: 8, time: 182
1/1 [==============================] - 0s 43ms/step
episode: 9, time: 152


In [9]:
#guardar el modelo entrenado, con los pesos
agent.model.save('modelo_cartpole_entrenado_relu.h5')
# Guardar el modelo en el formato nativo de Keras
agent.model.save('modelo_cartpole_entrenado', save_format='tf')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


**Si time = 200, signigica que el modelo tiene el 100% de éxito, porque a los 200 ciclos de tiempo, el juego se reinicia.**

Obtengo,finalmente episode: 24, time: 197